In [366]:
from tokenize import Number
import pyspark
import pyspark.sql.functions
from pyspark.sql.functions import round
from pyspark.sql.functions import regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import array_union
from pyspark.sql.functions import array
from pyspark.sql.functions import size
from pyspark.sql.functions import when


spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkLab2') \
                    .getOrCreate()

tiktokData2019 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2019.csv")
tiktokData2020 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2020.csv")
tiktokData2021 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2021.csv")
tiktokData2022 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2022.csv")

In [367]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import lit

df19 = tiktokData2019\
.select('artist_pop','track_pop','track_name', 'artist_name', 'danceability', 'album', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence')\
.withColumn('year', lit(2019))

df20 = tiktokData2020\
.select('artist_pop','track_pop','track_name', 'artist_name', 'danceability', 'album', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence')\
.withColumn('year', lit(2020))

df21 = tiktokData2021\
.select('artist_pop','track_pop','track_name', 'artist_name', 'danceability', 'album', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence')\
.withColumn('year', lit(2021))

df22 = tiktokData2022\
.select('artist_pop','track_pop','track_name', 'artist_name', 'danceability', 'album', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence')\
.withColumn('year', lit(2022))

df = df19.unionByName(df20).unionByName(df21).unionByName(df22)

df.select('artist_pop','track_pop','track_name', 'artist_name', 'danceability', 'album', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'year')

tiktokData = df.sort(desc("danceability")).select("track_name", "artist_name", "year", "danceability").show(6)


+--------------------+-----------+----+------------+
|          track_name|artist_name|year|danceability|
+--------------------+-----------+----+------------+
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|
+--------------------+-----------+----+------------+
only showing top 6 rows



In [368]:
from pyspark.sql.functions import col
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import count

df1 = df\

df1 = df.select('artist_name',"artist_pop",'track_name', 'album')\
.groupBy('artist_name','track_name', 'album')\
.agg(collect_set("artist_pop").alias('popularity'))\
.withColumn("other", size("popularity"))\
.where("other > 1")


df1.show()

+-------------+---------------+---------------+----------+-----+
|  artist_name|     track_name|          album|popularity|other|
+-------------+---------------+---------------+----------+-----+
|    Kesh Kesh|         Vibin'|         Vibin'|  [27, 26]|    2|
| Mahogany Lox|  Take Your Man|  Take Your Man|  [37, 38]|    2|
|Trevor Daniel|        Falling|        Falling|  [66, 65]|    2|
|    blackbear|hot girl bummer|hot girl bummer|  [79, 80]|    2|
+-------------+---------------+---------------+----------+-----+



In [369]:
from pyspark.sql.functions import array_max
from pyspark.sql.functions import when

df3 = df\

df2 = df1.select(col("track_name").alias("track_name1"), col("album").alias("album1"), 'artist_name', array_max("popularity").alias('artist_pop'))

df4 = df3.select("track_name", "artist_name", "album", "artist_pop")
df3 = df3.select("track_name", col("artist_name").alias("artist_name1"), "album", col("artist_pop").alias("artist_pop1"))

df3 = df3.join(df2, df3.artist_name1 == df2.artist_name, "right")\
.select("track_name", "artist_name", "album", "artist_pop")\


substractDf = df3\
    .subtract(df4)\
    .select("track_name", "artist_name", "album", "artist_pop")\
    .sort(desc("artist_pop"))

substractDf.show(6)
df3.count()

+----------+-------------+--------+----------+
|track_name|  artist_name|   album|artist_pop|
+----------+-------------+--------+----------+
|   Falling|Trevor Daniel|Nicotine|        66|
+----------+-------------+--------+----------+



14

In [370]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import max
from pyspark.sql.functions import avg
from pyspark.sql.functions import size
from pyspark.sql.functions import sort_array

df3 = df\

df3 = df.select('artist_pop','track_pop','track_name', 'album', 'artist_name', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'year')\
.groupBy('track_name','album' , 'artist_name', 'artist_pop')\
.agg(collect_list("year").alias('years'),
     max('track_pop').alias('max_track_pop'),
     avg('danceability').alias('avg_danceability'),
     avg('energy').alias('avg_energy'),
     avg('speechiness').alias('avg_speechiness'),
     avg('acousticness').alias('avg_acousticness'),
     avg('instrumentalness').alias('avg_instrumentalness'),
     avg('liveness').alias('avg_liveness'),
     avg('valence').alias('avg_valence'))\

df3.write.format("parquet").mode("overwrite").save("full_document")
df3.count()

853

In [371]:
df4 = df\

df4 = df.select('track_name', 'album' ,'artist_name','track_pop', 'year')\
.groupBy('artist_name','album','track_name','track_pop')\
.agg(collect_list("year").alias('years'))\
.sort(desc(size("years")), desc('track_pop'))

df4.show(6)


+----------------+--------------------+--------------------+---------+--------------------+
|     artist_name|               album|          track_name|track_pop|               years|
+----------------+--------------------+--------------------+---------+--------------------+
|        Doja Cat|            Hot Pink|              Say So|       80|[2020, 2021, 2022...|
|          WILLOW|        ARDIPITHECUS|      Wait a Minute!|       86|  [2020, 2022, 2019]|
|           BØRNS|            Dopamine|       Electric Love|       82|  [2020, 2021, 2022]|
|Melanie Martinez|Cry Baby (Deluxe ...|           Play Date|       78|  [2020, 2021, 2022]|
|  Arizona Zervas|             ROXANNE|             ROXANNE|       78|  [2019, 2020, 2022]|
|       Jawsh 685|Savage Love (Laxe...|Savage Love (Laxe...|       77|  [2020, 2021, 2022]|
+----------------+--------------------+--------------------+---------+--------------------+
only showing top 6 rows



In [372]:
df6 = df\

df6 = df.select("track_name", 'album' ,'artist_name','year')\
.groupBy("track_name")\
.agg(collect_list("track_name").alias('track_names'),
     collect_list("album").alias('albums'),
     collect_list('artist_name').alias('artist_names'),
    collect_list('year').alias('years'))\
.where(count("track_name") > 1)\
.select("track_names", 'albums' ,'artist_names','years')

df6.show()
df6.count()


+--------------------+--------------------+--------------------+------------------+
|         track_names|              albums|        artist_names|             years|
+--------------------+--------------------+--------------------+------------------+
|["More Than A Wom...|[Greatest, How Ca...|[Bee Gees, Bee Gees]|      [2020, 2022]|
|[223's (feat. 9lo...|[223's (feat. 9lo...|[YNW Melly, YNW M...|      [2020, 2019]|
|      [34+35, 34+35]|[Positions, Posit...|[Ariana Grande, A...|      [2021, 2022]|
|[Aesthetic, Aesth...|[Aesthetic, Aesth...|        [Xilo, Xilo]|      [2020, 2022]|
|[Airplane Mode, A...|        [Holo, Holo]|      [Limbo, Limbo]|      [2020, 2019]|
|[American Boy, Am...|      [Shine, Shine]|  [Estelle, Estelle]|      [2019, 2020]|
|[Another Day Of S...|[La La Land (Orig...|[La La Land Cast,...|      [2019, 2020]|
|[As It Was, As It...|[As It Was, As It...|[Michael Shynes, ...|      [2022, 2022]|
|[Astronaut In The...|[Astronaut In The...|[Masked Wolf, Mas...|      [2021,

145

+--------------------+--------------------+--------------------+------------------+
|         track_names|              albums|        artist_names|             years|
+--------------------+--------------------+--------------------+------------------+
|["More Than A Wom...|[Greatest, How Ca...|[Bee Gees, Bee Gees]|      [2020, 2022]|
|[223's (feat. 9lo...|[223's (feat. 9lo...|[YNW Melly, YNW M...|      [2020, 2019]|
|      [34+35, 34+35]|[Positions, Posit...|[Ariana Grande, A...|      [2021, 2022]|
|[Aesthetic, Aesth...|[Aesthetic, Aesth...|        [Xilo, Xilo]|      [2020, 2022]|
|[Airplane Mode, A...|        [Holo, Holo]|      [Limbo, Limbo]|      [2020, 2019]|
|[American Boy, Am...|      [Shine, Shine]|  [Estelle, Estelle]|      [2019, 2020]|
|[Another Day Of S...|[La La Land (Orig...|[La La Land Cast,...|      [2019, 2020]|
|[As It Was, As It...|[As It Was, As It...|[Michael Shynes, ...|      [2022, 2022]|
|[Astronaut In The...|[Astronaut In The...|[Masked Wolf, Mas...|      [2021,

145